In [106]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn import tree
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, f1_score, roc_curve, auc, accuracy_score, confusion_matrix, classification_report, roc_auc_score

from yellowbrick.classifier import ROCAUC
from yellowbrick.datasets import load_game

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 90)

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize 

In [41]:
df_p = pd.read_csv('data/random_full.csv')

In [42]:
df_r = pd.read_csv('data/full_working.csv')

In [43]:
df_p['genre'] = 'pop'
df_r['genre'] = 'reggae'

In [52]:
df = pd.concat([df_p, df_r])

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [53]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'acousticness', 'album', 'artist',
       'danceability', 'duration_ms', 'energy', 'explicit', 'genre',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'release_date', 'speechiness', 'tempo', 'time_signature', 'track_id',
       'track_name', 'track_uri', 'type', 'valence'],
      dtype='object')

In [54]:
df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'album', 'artist', 'track_id','track_name','type','release_date','track_id','track_name', 'track_uri',], inplace=True)

In [55]:
labelencoder = LabelEncoder()

df['genre'] = labelencoder.fit_transform(df['genre'])
df['explicit'] = labelencoder.fit_transform(df['explicit'])

In [56]:
df['explicit']= df['explicit'].astype('category')

In [57]:
df = pd.get_dummies(df)

In [58]:
df

,acousticness,danceability,duration_ms,energy,genre,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,explicit_0,explicit_1
0,0.01250,0.794,183290.0,0.793,0,0.000000,11.0,0.0952,-4.521,0.0,0.0842,123.941,4.0,0.677,1,0
1,0.60400,0.459,215107.0,0.575,0,0.000000,3.0,0.0885,-4.858,1.0,0.0573,111.881,4.0,0.183,1,0
2,0.12900,0.598,168387.0,0.526,0,0.000000,10.0,0.1400,-8.659,0.0,0.0415,123.935,4.0,0.529,1,0
3,0.00838,0.817,216707.0,0.781,0,0.000005,7.0,0.1230,-4.178,0.0,0.1120,104.000,4.0,0.359,1,0
4,0.40400,0.662,210427.0,0.519,0,0.000000,9.0,0.1210,-6.554,0.0,0.1060,145.841,4.0,0.495,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4833,0.09950,0.754,245701.0,0.932,1,0.000419,8.0,0.2010,-5.073,1.0,0.0590,130.017,4.0,0.851,1,0
4834,0.09310,0.642,136385.0,0.853,1,0.000000,6.0,0.1180,-4.059,1.0,0.2790,113.931,4.0,0.811,1,0
4835,0.05310,0.823,174405.0,0.653,1,0.000080,4.0,0.0904,-4.378,0.0,0.0402,100.023,4.0,0.588,1,0
4836,0.08690,0.833,159048.0,0.782,1,0.000000,7.0,0.1100,-3.743,1.0,0.2810,157.037,4.0,0.820,1,0


In [59]:
df.genre.value_counts()

1    4838
0    1362
Name: genre, dtype: int64

# Classifier

In [66]:
df.dropna(inplace=True)

In [67]:
y = df['genre']
X = df.drop(columns=['genre'], axis=1)

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state=123)

In [117]:
pipe_knn = Pipeline([('scl', StandardScaler()),
                    ('pca', PCA(n_components=5)),
                    ('knn', KNeighborsClassifier(weights='uniform'))])

param_grid_knn = {'knn__n_neighbors':[15,20,25,30] ,
                  'knn__leaf_size':[20,30,40,50],
                  'knn__algorithm':['auto', 'ball_tree', 'kd_tree', 'brute']
                    }

gs_knn = GridSearchCV(pipe_knn, param_grid_knn, cv=5)

gs_knn.fit(X_train, y_train)

gs_knn_best = gs_knn.best_params_
print(gs_knn_best)

{'knn__algorithm': 'auto', 'knn__leaf_size': 20, 'knn__n_neighbors': 30}


###### Decision tree cannot be optimised beyond basic model

In [118]:
pipe_dt = Pipeline([('scl', StandardScaler()),
                    ('tree', tree.DecisionTreeClassifier(class_weight='balanced', random_state=123))])

param_grid_dt = {'tree__max_features':[6,8,10,12,14],
                 'tree__min_samples_leaf':[5,10,15],
                 'tree__max_depth':[6,7,8,9]  
             }

gs_dt = GridSearchCV(pipe_dt, param_grid_dt, cv=5)

gs_dt.fit(X_train, y_train)
gs_dt.predict(X_test)

gs_dt_best = gs_dt.best_params_

print(gs_dt_best)

{'tree__max_depth': 6, 'tree__max_features': 12, 'tree__min_samples_leaf': 5}


In [119]:
pipe_lr_1 = Pipeline([('scl', StandardScaler()),
                    ('pca', PCA(n_components=5)),
                    ('log', LogisticRegression(class_weight='balanced', random_state=123, solver='saga'))])

param_grid_lr = {'log__penalty':['elasticnet'],
                'log__max_iter':[100,200,300,400,500],
                'log__l1_ratio':[0.05, 0.1, 0.15, 0.2,0.4,0.6]
                 }

gs_lr_1 = GridSearchCV(pipe_lr_1, param_grid_lr, cv=5)

gs_lr_1.fit(X_train, y_train)

gs_lr_1_best = gs_lr_1.best_params_

print(gs_lr_1_best)

{'log__l1_ratio': 0.05, 'log__max_iter': 100, 'log__penalty': 'elasticnet'}


In [120]:
pipe_lr_2 = Pipeline([('scl', StandardScaler()),
                    ('pca', PCA(n_components=5)),
                    ('log', LogisticRegression(random_state=123, class_weight='balanced'))])

param_grid_lr = {'log__penalty':['l2'],
                 'log__solver':['newton-cg','sag','lbfgs'],
                 'log__max_iter':[75,100,125,150]
                 }

gs_lr_2 = GridSearchCV(pipe_lr_2, param_grid_lr, cv=5)

gs_lr_2.fit(X_train, y_train)

gs_lr_2_best = gs_lr_2.best_params_

print(gs_lr_2_best)

{'log__max_iter': 75, 'log__penalty': 'l2', 'log__solver': 'newton-cg'}


In [121]:
pipe_bt = Pipeline([('scl', StandardScaler()),
                    ('bt', BaggingClassifier(tree.DecisionTreeClassifier(random_state = 123), random_state=123))])


param_grid_bt = {'bt__base_estimator__max_features':[6,8,10,12,14],
                 'bt__base_estimator__min_samples_leaf':[5,10,15],
                 'bt__base_estimator__max_depth':[10,11,12,13,14],
                 'bt__n_estimators':[10,12,14,16,18,20]
             }

gs_bt = GridSearchCV(pipe_bt, param_grid_bt, cv=5)

gs_bt.fit(X_train, y_train)
gs_bt.predict(X_test)

gs_bt_best = gs_bt.best_params_

print(gs_bt_best)

{'bt__base_estimator__max_depth': 13, 'bt__base_estimator__max_features': 10, 'bt__base_estimator__min_samples_leaf': 5, 'bt__n_estimators': 18}


In [122]:
pipe_bo = Pipeline([('scl', StandardScaler()),
                    ('boo', AdaBoostClassifier(tree.DecisionTreeClassifier(random_state=123)))])


param_grid_bo = {'boo__base_estimator__max_features':[5,10,15],
                 'boo__base_estimator__min_samples_leaf':[5,7,9],
                 'boo__base_estimator__max_depth':[10,11,12,13,14],
                 'boo__n_estimators':[14,16,18,20]
                 
             }

gs_bo = GridSearchCV(pipe_bo, param_grid_bo, cv=5)

gs_bo.fit(X_train, y_train)
gs_bo.predict(X_test)

gs_bo_best = gs_bo.best_params_

print(gs_bt_best)

{'bt__base_estimator__max_depth': 13, 'bt__base_estimator__max_features': 10, 'bt__base_estimator__min_samples_leaf': 5, 'bt__n_estimators': 18}


In [109]:
estimator = pipe_bo
estimator.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'scl', 'boo', 'scl__copy', 'scl__with_mean', 'scl__with_std', 'boo__algorithm', 'boo__base_estimator__class_weight', 'boo__base_estimator__criterion', 'boo__base_estimator__max_depth', 'boo__base_estimator__max_features', 'boo__base_estimator__max_leaf_nodes', 'boo__base_estimator__min_impurity_decrease', 'boo__base_estimator__min_impurity_split', 'boo__base_estimator__min_samples_leaf', 'boo__base_estimator__min_samples_split', 'boo__base_estimator__min_weight_fraction_leaf', 'boo__base_estimator__presort', 'boo__base_estimator__random_state', 'boo__base_estimator__splitter', 'boo__base_estimator', 'boo__learning_rate', 'boo__n_estimators', 'boo__random_state'])

In [126]:
pipes = [gs_lr_1, gs_lr_2, gs_dt, gs_bt, gs_knn, gs_bo]
pipe_dict = {0: 'Log Reg Elastic', 1: 'Log Reg Laso', 2: 'Decision Tree', 3: 'Bagged Tree', 4: 'K Nearest', 5: 'AdaBoost'}

def best_pipe(pipes, pipe_dict):
    for pipe in pipes:
        pip = pipe.best_estimator_
        pip.fit(X_train, y_train)

        # # Compare accuracies
    for idx, val in enumerate(pipes):
         print('%s pipeline test accuracy: %.4f' % (pipe_dict[idx], val.score(X_test, y_test)))

    # Identify the most accurate model on test data
    best_acc = 0.0
    best_clf = 0
    best_pipe = ''
    for idx, val in enumerate(pipes):
        if val.score(X_test, y_test) > best_acc:
            best_acc = val.score(X_test, y_test)
            best_pipe = val
            best_clf = idx

    # Save pipeline to file
    joblib.dump(best_pipe, 'binary.pkl', compress=1)
    print('Saved %s pipeline to file' % pipe_dict[best_clf])
  
best_pipe(pipes, pipe_dict)

Log Reg Elastic pipeline test accuracy: 0.7315
Log Reg Laso pipeline test accuracy: 0.7315
Decision Tree pipeline test accuracy: 0.8153
Bagged Tree pipeline test accuracy: 0.8750
K Nearest pipeline test accuracy: 0.8202
AdaBoost pipeline test accuracy: 0.8661
Saved Bagged Tree pipeline to file
